In [1]:
'''An implementation of sequence to sequence learning for performing addition
        Input: "535+61"
        Output: "596"

Padding is handled by using a repeated sentinel character (space)
Input may optionally be inverted, shown to increase performance in many tasks in:
        "Learning to Execute" 
            http://arxiv.org/abs/1410.4615 and
        "Sequence to Sequence Learning with Neural Networks" 
            http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Two digits inverted:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs
Three digits inverted:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs
Four digits inverted:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs
Five digits inverted:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''

from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


In [2]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
INVERT = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []

In [3]:
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    
    seen.add(key)
    
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    
    if INVERT:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 50000
Vectorization...


In [4]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


In [5]:
# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()

# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))

# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))

# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if INVERT else q)
        print('T', correct)
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=" ")
        else:
            print(colors.fail + '☒' + colors.close, end=" ")
        print(guess)
        print('---')

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________

--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/

45000/45000 [==============================] - 40s - loss: 0.3459 - acc: 0.8937 - val_loss: 0.3122 - val_acc: 0.9039
Q 181+227
T 408 
☒ 308 
---
Q 829+366
T 1195
☒ 1185
---
Q 72+140 
T 212 
☑ 212 
---
Q 312+211
T 523 
☒ 423 
---
Q 3+530  
T 533 
☒ 433 
---
Q 334+694
T 1028
☑ 1028
---
Q 3+31   
T 34  
☑ 34  
---
Q 64+933 
T 997 
☒ 998 
---
Q 8+969  
T 977 
☑ 977 
---
Q 231+244
T 475 
☒ 476 
---

--------------------------------------------------
Iteration 14
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 41s - loss: 0.2491 - acc: 0.9356 - val_loss: 0.2204 - val_acc: 0.9428
Q 237+540
T 777 
☒ 776 
---
Q 659+513
T 1172
☑ 1172
---
Q 16+632 
T 648 
☑ 648 
---
Q 603+7  
T 610 
☒ 600 
---
Q 85+582 
T 667 
☑ 667 
---
Q 65+946 
T 1011
☑ 1011
---
Q 359+41 
T 400 
☒ 300 
---
Q 68+12  
T 80  
☒ 70  
---
Q 569+48 
T 617 
☑ 617 
---
Q 67+17  
T 84  
☑ 84  
---

--------------------------------------------------
Iteration 15
Train on 45000 sa

45000/45000 [==============================] - 31s - loss: 0.0489 - acc: 0.9864 - val_loss: 0.0233 - val_acc: 0.9952
Q 661+5  
T 666 
☑ 666 
---
Q 201+58 
T 259 
☑ 259 
---
Q 985+905
T 1890
☑ 1890
---
Q 94+17  
T 111 
☑ 111 
---
Q 531+79 
T 610 
☑ 610 
---
Q 45+759 
T 804 
☑ 804 
---
Q 30+481 
T 511 
☑ 511 
---
Q 895+8  
T 903 
☑ 903 
---
Q 315+835
T 1150
☑ 1150
---
Q 645+6  
T 651 
☑ 651 
---

--------------------------------------------------
Iteration 28
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 30s - loss: 0.0126 - acc: 0.9990 - val_loss: 0.0179 - val_acc: 0.9968
Q 371+24 
T 395 
☑ 395 
---
Q 188+83 
T 271 
☑ 271 
---
Q 61+916 
T 977 
☑ 977 
---
Q 904+854
T 1758
☑ 1758
---
Q 629+530
T 1159
☑ 1159
---
Q 662+765
T 1427
☑ 1427
---
Q 981+225
T 1206
☑ 1206
---
Q 568+7  
T 575 
☑ 575 
---
Q 829+68 
T 897 
☑ 897 
---
Q 258+9  
T 267 
☑ 267 
---

--------------------------------------------------
Iteration 29
Train on 45000 sa

45000/45000 [==============================] - 31s - loss: 0.0029 - acc: 0.9999 - val_loss: 0.0062 - val_acc: 0.9990
Q 408+511
T 919 
☑ 919 
---
Q 6+133  
T 139 
☑ 139 
---
Q 30+204 
T 234 
☑ 234 
---
Q 140+386
T 526 
☑ 526 
---
Q 84+19  
T 103 
☑ 103 
---
Q 96+595 
T 691 
☑ 691 
---
Q 40+729 
T 769 
☑ 769 
---
Q 40+69  
T 109 
☑ 109 
---
Q 238+39 
T 277 
☑ 277 
---
Q 841+225
T 1066
☑ 1066
---

--------------------------------------------------
Iteration 42
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 30s - loss: 0.0292 - acc: 0.9915 - val_loss: 0.1390 - val_acc: 0.9573
Q 158+35 
T 193 
☑ 193 
---
Q 935+12 
T 947 
☑ 947 
---
Q 556+0  
T 556 
☑ 556 
---
Q 590+237
T 827 
☑ 827 
---
Q 487+230
T 717 
☑ 717 
---
Q 16+656 
T 672 
☑ 672 
---
Q 80+64  
T 144 
☑ 144 
---
Q 449+902
T 1351
☑ 1351
---
Q 989+742
T 1731
☒ 1721
---
Q 19+862 
T 881 
☑ 881 
---

--------------------------------------------------
Iteration 43
Train on 45000 sa

45000/45000 [==============================] - 30s - loss: 0.0179 - acc: 0.9948 - val_loss: 0.0065 - val_acc: 0.9984
Q 210+114
T 324 
☑ 324 
---
Q 41+886 
T 927 
☑ 927 
---
Q 801+29 
T 830 
☑ 830 
---
Q 52+669 
T 721 
☑ 721 
---
Q 31+131 
T 162 
☑ 162 
---
Q 873+27 
T 900 
☑ 900 
---
Q 718+77 
T 795 
☑ 795 
---
Q 34+174 
T 208 
☑ 208 
---
Q 99+298 
T 397 
☑ 397 
---
Q 315+823
T 1138
☑ 1138
---

--------------------------------------------------
Iteration 56
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 31s - loss: 0.0020 - acc: 0.9999 - val_loss: 0.0049 - val_acc: 0.9990
Q 704+14 
T 718 
☑ 718 
---
Q 892+964
T 1856
☑ 1856
---
Q 888+74 
T 962 
☑ 962 
---
Q 82+211 
T 293 
☑ 293 
---
Q 67+17  
T 84  
☑ 84  
---
Q 72+628 
T 700 
☑ 700 
---
Q 93+786 
T 879 
☑ 879 
---
Q 8+450  
T 458 
☑ 458 
---
Q 25+73  
T 98  
☑ 98  
---
Q 51+630 
T 681 
☑ 681 
---

--------------------------------------------------
Iteration 57
Train on 45000 sa

...

45000/45000 [==============================] - 17s - loss: 1.6911e-04 - acc: 1.0000 - val_loss: 0.0014 - val_acc: 0.9997
Q 70+320 
T 390 
☑ 390 
---
Q 33+835 
T 868 
☑ 868 
---
Q 203+458
T 661 
☑ 661 
---
Q 793+1  
T 794 
☑ 794 
---
Q 611+37 
T 648 
☑ 648 
---
Q 167+4  
T 171 
☑ 171 
---
Q 83+625 
T 708 
☑ 708 
---
Q 511+877
T 1388
☑ 1388
---
Q 24+226 
T 250 
☑ 250 
---
Q 41+721 
T 762 
☑ 762 
---

--------------------------------------------------
Iteration 140
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 17s - loss: 1.5227e-04 - acc: 1.0000 - val_loss: 0.0013 - val_acc: 0.9996
Q 94+17  
T 111 
☑ 111 
---
Q 469+29 
T 498 
☑ 498 
---
Q 33+490 
T 523 
☑ 523 
---
Q 9+925  
T 934 
☑ 934 
---
Q 650+502
T 1152
☑ 1152
---
Q 231+412
T 643 
☑ 643 
---
Q 124+25 
T 149 
☑ 149 
---
Q 589+83 
T 672 
☑ 672 
---
Q 228+933
T 1161
☑ 1161
---
Q 611+92 
T 703 
☑ 703 
---

--------------------------------------------------
Iteration 141
Train o

45000/45000 [==============================] - 17s - loss: 1.1862e-04 - acc: 1.0000 - val_loss: 0.0011 - val_acc: 0.9997
Q 617+695
T 1312
☑ 1312
---
Q 82+667 
T 749 
☑ 749 
---
Q 794+41 
T 835 
☑ 835 
---
Q 335+334
T 669 
☑ 669 
---
Q 55+21  
T 76  
☑ 76  
---
Q 0+871  
T 871 
☑ 871 
---
Q 888+40 
T 928 
☑ 928 
---
Q 772+545
T 1317
☑ 1317
---
Q 113+795
T 908 
☑ 908 
---
Q 31+473 
T 504 
☑ 504 
---

--------------------------------------------------
Iteration 154
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 17s - loss: 1.1075e-04 - acc: 1.0000 - val_loss: 0.0013 - val_acc: 0.9997
Q 241+813
T 1054
☑ 1054
---
Q 55+829 
T 884 
☑ 884 
---
Q 754+662
T 1416
☑ 1416
---
Q 602+810
T 1412
☑ 1412
---
Q 22+35  
T 57  
☑ 57  
---
Q 37+877 
T 914 
☑ 914 
---
Q 703+31 
T 734 
☑ 734 
---
Q 78+780 
T 858 
☑ 858 
---
Q 940+948
T 1888
☑ 1888
---
Q 782+319
T 1101
☑ 1101
---

--------------------------------------------------
Iteration 155
Train o

45000/45000 [==============================] - 17s - loss: 0.0105 - acc: 0.9974 - val_loss: 0.2101 - val_acc: 0.9476
Q 90+277 
T 367 
☑ 367 
---
Q 340+0  
T 340 
☑ 340 
---
Q 366+4  
T 370 
☑ 370 
---
Q 535+993
T 1528
☑ 1528
---
Q 60+982 
T 1042
☑ 1042
---
Q 771+867
T 1638
☒ 1538
---
Q 9+375  
T 384 
☑ 384 
---
Q 769+26 
T 795 
☑ 795 
---
Q 894+22 
T 916 
☑ 916 
---
Q 80+6   
T 86  
☑ 86  
---

--------------------------------------------------
Iteration 168
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 17s - loss: 0.0299 - acc: 0.9915 - val_loss: 0.0025 - val_acc: 0.9993
Q 897+913
T 1810
☑ 1810
---
Q 434+717
T 1151
☑ 1151
---
Q 16+62  
T 78  
☑ 78  
---
Q 526+2  
T 528 
☑ 528 
---
Q 29+429 
T 458 
☑ 458 
---
Q 2+823  
T 825 
☑ 825 
---
Q 533+53 
T 586 
☑ 586 
---
Q 32+10  
T 42  
☑ 42  
---
Q 287+9  
T 296 
☑ 296 
---
Q 21+125 
T 146 
☑ 146 
---

--------------------------------------------------
Iteration 169
Train on 45000 

45000/45000 [==============================] - 17s - loss: 0.0324 - acc: 0.9917 - val_loss: 0.0058 - val_acc: 0.9983
Q 190+97 
T 287 
☑ 287 
---
Q 129+999
T 1128
☑ 1128
---
Q 952+351
T 1303
☑ 1303
---
Q 390+461
T 851 
☑ 851 
---
Q 68+258 
T 326 
☑ 326 
---
Q 278+349
T 627 
☑ 627 
---
Q 568+7  
T 575 
☑ 575 
---
Q 58+589 
T 647 
☑ 647 
---
Q 425+31 
T 456 
☑ 456 
---
Q 9+508  
T 517 
☑ 517 
---

--------------------------------------------------
Iteration 182
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 17s - loss: 0.0013 - acc: 0.9998 - val_loss: 0.0016 - val_acc: 0.9996
Q 713+614
T 1327
☑ 1327
---
Q 113+271
T 384 
☑ 384 
---
Q 74+658 
T 732 
☑ 732 
---
Q 81+26  
T 107 
☑ 107 
---
Q 270+70 
T 340 
☑ 340 
---
Q 91+759 
T 850 
☑ 850 
---
Q 382+13 
T 395 
☑ 395 
---
Q 1+286  
T 287 
☑ 287 
---
Q 363+75 
T 438 
☑ 438 
---
Q 184+223
T 407 
☑ 407 
---

--------------------------------------------------
Iteration 183
Train on 45000 

45000/45000 [==============================] - 18s - loss: 0.0310 - acc: 0.9916 - val_loss: 0.0048 - val_acc: 0.9986
Q 810+55 
T 865 
☑ 865 
---
Q 717+747
T 1464
☑ 1464
---
Q 37+532 
T 569 
☑ 569 
---
Q 90+238 
T 328 
☑ 328 
---
Q 929+424
T 1353
☑ 1353
---
Q 54+95  
T 149 
☑ 149 
---
Q 884+34 
T 918 
☑ 918 
---
Q 57+72  
T 129 
☑ 129 
---
Q 138+335
T 473 
☑ 473 
---
Q 207+86 
T 293 
☑ 293 
---

--------------------------------------------------
Iteration 196
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 17s - loss: 0.0013 - acc: 0.9998 - val_loss: 0.0015 - val_acc: 0.9997
Q 489+197
T 686 
☑ 686 
---
Q 81+14  
T 95  
☑ 95  
---
Q 94+17  
T 111 
☑ 111 
---
Q 435+84 
T 519 
☑ 519 
---
Q 88+292 
T 380 
☑ 380 
---
Q 5+961  
T 966 
☑ 966 
---
Q 43+591 
T 634 
☑ 634 
---
Q 39+345 
T 384 
☑ 384 
---
Q 942+112
T 1054
☑ 1054
---
Q 90+277 
T 367 
☑ 367 
---

--------------------------------------------------
Iteration 197
Train on 45000 